In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [50]:
#Dropping the nonsense rows at the beginning and specifying the column row
data = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls',
                    header=1, skiprows=[0,1,2,])

In [51]:
data.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861.000,0.000,0.000,nan,0.000,0.000,0.000,12.000,2.000,10.000,0.000,0.000
1,Addison Town and Village,2577.000,3.000,0.000,nan,0.000,0.000,3.000,24.000,3.000,20.000,1.000,0.000
2,Akron Village,2846.000,3.000,0.000,nan,0.000,0.000,3.000,16.000,1.000,15.000,0.000,0.000
3,Albany,97956.000,791.000,8.000,nan,30.000,227.000,526.000,4090.000,705.000,3243.000,142.000,nan
4,Albion Village,6388.000,23.000,0.000,nan,3.000,4.000,16.000,223.000,53.000,165.000,5.000,nan


In [62]:
#Only selecting columns needed for evaluation
data_short = data.iloc[:, [1,3,6]]

In [63]:
data_short.head()

,Population,Murder and nonnegligent manslaughter,Robbery
0,1861.000,0.000,0.000
1,2577.000,0.000,0.000
2,2846.000,0.000,0.000
3,97956.000,8.000,227.000
4,6388.000,0.000,4.000


In [64]:
#Adding Pop squared column
data_short['Population_sq'] = data_short.Population ** 2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
#Renaming columns, because of strange error
data_short.columns = ['Population', 'Murder','Robbery', 'Population_sq']

In [72]:
#Turning Murder continuous into categorical
data_short.Murder = (data_short.Murder > 0).astype(int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [74]:
data_short.Murder.head()

0    0
1    0
2    0
3    1
4    0
Name: Murder, dtype: int64

In [75]:
#Turning Robbery continuous into categorical
data_short.Robbery = (data_short.Robbery > 0).astype(int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [76]:
data_short.Robbery.head()

0    0
1    0
2    0
3    1
4    1
Name: Robbery, dtype: int64

In [86]:
#Checking if we have NaN values in the data set
data_check = data_short.apply(lambda x: np.isnan(x))

In [90]:
#Looks like we do for the two population variables
for item in data_check:
    display(data_check[item].value_counts())

False    348
True       3
Name: Population, dtype: int64

False    351
Name: Murder, dtype: int64

False    351
Name: Robbery, dtype: int64

False    348
True       3
Name: Population_sq, dtype: int64

In [97]:
#Removing the NaN
data_short = data_short.dropna()

In [98]:
#Checking that all the values are above zero
(data_short.Population > 0).astype(int).sum()

348